In [16]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection
import ccdproc as ccdp

from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u
from convenience_functions import show_image

# suppress warnings is not recommended :I
import warnings
warnings.filterwarnings("ignore")

bias_dir = './data/bias'
flat_dir = './data/flat'
darks_dir = './data/dark'
tzboo_dir = './data/tz_boo'
cccom_dir = './data/cc_com'

# Responses to the Data Reduction & Photometry Lab
## 1. Define the bias. Include both a description of what it looks like physically, why it occurs, and how we take it into account (how do we remove its effect on our data images?)

A bias contains a grid of values added to every pixel on the CCD to ensure no negative pixel values are output. A bias value to a pixel can be likened to adjusting the y-intercept of a linear function. We expect to account for negative pixel counts because reading noise can be modeled as a Gaussian distribution with a mean of zero and some standard deviation. The read noise comes from material imperfections of the CCD. Ideally, a bias would be uniform across all CCD pixels.

In practice, biases are offset voltages. To take a bias of a given CCD instrument, one reads out the CCD with zero exposure time. 

## 2. Why would we use more than one bias frame?

The noise reduction factor goes like $\sim 1/\sqrt{N}$, with $N$ as the number of combined biases to be combined. Having multiple biases thus increases the amount the noise is reduced

## 3. List the average value of each of your bias frames, as well as your final master bias.

In [ ]:
biases = ImageFileCollection(bias_dir).files_filtered(imagetyp='bias', include_path=True)

combined_bias = ccdp.combine(biases,
                             method='average',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std,
                             mem_limit=350e6,
                             unit='adu'
                            );
combined_bias.write(bias_dir + '/combined_bias.fits')

In [26]:
reduced_biases = ImageFileCollection(bias_dir).files_filtered(imagetyp='bias', include_path=True)
for bias in reduced_biases:
    mean = CCDData.read(bias, unit='adu').data.mean()
    print(f'mean for {bias}: {mean:.3f}')
    
    
combined_bias = CCDData.read(bias_dir + '/combined_bias.fits')
print('combined mean: ' + str(combined_bias.data.mean()))

mean for ./data/bias/Bias_BIN1_20230424_031732.fits: 980.423
mean for ./data/bias/Bias_BIN1_20230424_031823.fits: 980.209
mean for ./data/bias/Bias_BIN1_20230424_031904.fits: 980.381
mean for ./data/bias/Bias_BIN1_20230424_031946.fits: 980.619
mean for ./data/bias/Bias_BIN1_20230424_032028.fits: 980.514
mean for ./data/bias/Bias_BIN1_20230424_032109.fits: 980.559
mean for ./data/bias/Bias_BIN1_20230424_032152.fits: 980.830


INFO:astropy:using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
mean for ./data/bias/combined_bias.fits: 980.505
combined mean: 980.5052719468449


## 4. Define “overscan”

An overscan is the covered portion of the CCD sensor. This feature is useful in removing variations in the bias on a frame-by-frame basis. Data from the overscan includes bias, read noise, and dark current.

## 5. List your overscan parameters. (Where is the overscan on your images?)

ARCSAT images have no overscan.

## 6. Which filters do you have flat fields for? List the flat field file names for each filter that you need. Do you have more flats than you need? Confirm that your flat fields have the same binning as your data.

Our project took flat fields for B and V filters. Both observation images and flats have 1x1 binning. The files include:

In [ ]:
flats = ImageFileCollection(flat_dir)
flats.summary['file', 'xbinning', 'ybinning']

In [ ]:
tz_boo = ImageFileCollection(tzboo_dir)
tz_boo.summary['file', 'xbinning', 'ybinning']

In [ ]:
cc_com = ImageFileCollection(cccom_dir)
cc_com.summary['file', 'xbinning', 'ybinning']

## 7. Why do we need flat fields for each filter we have used for observations?
Flats involve reading data taken from a controlled source of light. They are taken to correct for visual artifacts like vignetting, dust on the CCD, etc. We need to make flat fields for each filter as the CCD sensor's response depends on the wavelength of incident photons.

- BIAS SUBTRACTION AND FLAT FIELD NORMALIZATION SHOULD BE DONE ON EACH OF
YOUR OBJECT IMAGE FILES. It is a good place to take a little time and develop a loop, or a
small pseudo pipeline

- To do: flat field normalization

## 8. What drawbacks could exist from this method of sky subtraction?
Determining the cutoff brightness for what pixel is considered a part of a star's light or the sky background brightness is not well-defined. If the cutoff brightness for the sky is too high, one might not account for all the light coming from a star. In the opposite case, too low of a cutoff brightness may overestimate the light coming from a star.

## 9. Why does sky subtraction matter?
Sky subtraction matters because the contributions of light from the background sky can add to the counts of a star one intends to measure. Not accounting for the contributions due to the background sky can cause overestimations in the brightness of a star.